## Importing libs and setting plotting parameters

In [7]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import timedelta, datetime

In [8]:
df = pd.read_parquet('../data/train.parquet')

## Organizing db to optimize the queries speed.

#### Here the idea is to generate a DataFrame containing the constant atributes of a loan where each row is going to represent a loan from a single company. Sequentialy, a dictionary is going to be generate where the key is the id of the loan and the value is a DataFrame with the history of the loan. This operation optimizes the generation of graphs for the dashboard.

In [14]:
%%time
#this loop takes approximately 4 mins to run
id_column_name = ['id']
constant_attributes = ['desembolso', 'vencimento', 'valor_emprestado', 'pgto_diario_esperado', 'subsegmento', 'y']
timeseries_attributes = ['dia', 'dias_pos_desembolso', 'divida_total', 'divida_principal', 'pagamento_diario', 'amortizacao_principal_diario',
                        'transacionado']

unique_index = np.unique(df[id_column_name])
unique_index.sort()

df_constant_attributes = pd.DataFrame(
    columns=constant_attributes, index=unique_index.astype(int)
)

company_ids_in_df = df[id_column_name].to_numpy()
history_loan = dict.fromkeys(set(unique_index))
history_loan.update(dict(history_loan))

for idx in unique_index:
    idx_boolean_list = company_ids_in_df == idx
    tmp_df = df.loc[idx_boolean_list]
    df_constant_attributes.loc[idx] = tmp_df[constant_attributes].iloc[-1]
    history_loan[idx] = tmp_df[timeseries_attributes].reset_index(drop=True)

CPU times: user 4min 13s, sys: 159 ms, total: 4min 14s
Wall time: 4min 14s


In [15]:
# null values in subsegmento are filled with `N/A`
df_constant_attributes['subsegmento'] = df_constant_attributes['subsegmento'].fillna('N/A')

# index of the constant dataframe is set to id
df_constant_attributes.index = df.id.astype(int)
df_constant_attributes.drop(columns='id', inplace=True)

### Building App

In [44]:
app = JupyterDash(__name__, assets_folder='../assets')

app.layout = html.Div([
    html.Div(id='header-info', className='header-info'),
    dcc.Dropdown(
        id='company-id', clearable=False,
        value=unique_index[0], options=[{'label': f'Loan ID: {index}', 'value': index}
                                        for index in unique_index], className='dropdown-select-id'
    ),
    html.Div([
        dcc.Graph(id='graph-debt', className='graph-id'),
        dcc.Graph(id='graph-transaction-payment', className='graph-id')
    ], className='container-graph-id'),

    html.Div(id='table-const-id', className='table-hidden'),
    html.Div(id='table-timeseries-id', className='table-hidden'),
])


@app.callback(
    Output('table-const-id', 'children'),
    Output('table-timeseries-id', 'children'),
    Input('company-id', 'value')
)
def update_table(loan_id):
    id_tmp_db_const = df_constant_attributes.loc[loan_id]
    id_tmp_db_timeseries = history_loan[loan_id]

    return id_tmp_db_const.to_json(date_format='iso', orient='split'), id_tmp_db_timeseries.to_json(date_format='iso', orient='split')


def create_time_series(df, title):
    fig = px.line(df, x='dia', y='value', color='variable', title=title)
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(title='Reais (R$)', visible=True, showticklabels=True)
    fig.update_xaxes(title='', visible=True, showticklabels=True)
    fig.update_layout(legend_orientation='h')
    fig.update_layout(legend_title_text='')
    fig.update_layout(height=300, margin={'l': 30, 'b': 30, 'r': 30, 't': 30})

    day_first, _ = df['dia'].iloc[0].split('T')
    day_ninety = datetime.strptime(day_first, '%Y-%m-%d') + timedelta(days=90)

    fig.add_vline(x=day_ninety.timestamp() * 1000, line_width=3, line_dash="dash",
                  line_color="green", annotation_text="90th day", annotation_position="top right")

    return fig


@app.callback(
    Output('graph-debt', 'figure'),
    Output('graph-transaction-payment', 'figure'),
    Input('table-timeseries-id', 'children')
)
def update_figure(jsonified_db):
    tmp_db = pd.read_json(jsonified_db, orient='split')

    tmp_db_debt = pd.melt(tmp_db[['dia', 'divida_principal', 'divida_total']],
                          id_vars=['dia'],
                          value_vars=['divida_principal', 'divida_total'])

    tmp_db_payment = pd.melt(tmp_db[['dia', 'transacionado', 'pagamento_diario', 'amortizacao_principal_diario']],
                             id_vars=['dia'],
                             value_vars=['transacionado', 'pagamento_diario', 'amortizacao_principal_diario'])

    return create_time_series(tmp_db_debt, title="Loan Debt"), create_time_series(tmp_db_payment, title="Transactions and Debt Payment")


@app.callback(
    Output('header-info', 'children'),
    Input('table-const-id', 'children')
)
def update_headers(jsonified_db):
    tmp_db = pd.read_json(jsonified_db, orient='split', typ='series')
    loan_contract, _ = tmp_db['desembolso'].split('T')
    loan_overdue, _ = tmp_db['vencimento'].split('T')
    subsegment = tmp_db['subsegmento']
    valor_emprestado = tmp_db['valor_emprestado']
    pgto_diario_esperado = tmp_db['pgto_diario_esperado']
    default = 'Sim' if tmp_db['y'] else 'Nao'
    return [
        html.Div([
            html.H4('Company Sector'),
            html.H3(f'{subsegment}')], className='info-cards'),
        html.Div([
            html.H4('Default'),
            html.H3(f'{default}')], className='info-cards'),
        html.Div([
            html.H4('Loan Value'),
            html.H3(f'{valor_emprestado}')], className='info-cards'),
        html.Div([
            html.H4('Loan Contract'),
            html.H3(f'{loan_contract}')], className='info-cards'),
        html.Div([
            html.H4('Expected Loan Over Due'),
            html.H3(f'{loan_overdue}')], className='info-cards'),
        html.Div([
            html.H4('Expected Loan Daily Payment'),
            html.H3(f'R$ {pgto_diario_esperado:.2f}')], className='info-cards')]


# Run app and display result inline in the notebook
app.run_server(mode='inline', port=8080, debug=True)